In [1]:
import itertools
from shapely.geometry import box
import warnings
import numpy as np
import geopandas as gpd
warnings.filterwarnings('ignore', 'Geometry is in a ', UserWarning)
from ml4floods.data import utils
import os

## Create grid

In [2]:
# step = 0.20, 0.20
# size = 0.21

# bounds_inicial = (146.86329100021769, -37.50516044138397, 153.63872785102905, -23.244936010554465)

# pols_all = []
# for idx, (x_left,y_bottom) in enumerate(itertools.product(np.arange(bounds_inicial[0]-10, bounds_inicial[2], step[0]),
#                                                           np.arange(bounds_inicial[1]-2,bounds_inicial[3]+13, step[1]))):
#     bounds_iter = (x_left, y_bottom, x_left+size, y_bottom+size)
#     pol = box(*bounds_iter)
#     pols_all.append({"geometry": pol, "name":f"GRID{idx:05d}"})

# pols_all = gpd.GeoDataFrame(pols_all,crs="EPSG:4326")
# pols_all.to_file("/home/gonzalo/Downloads/grid_australia.geojson",driver="GeoJSON")  
# print(pols_all.shape)

# gpd.GeoDataFrame(geometry=[box(*pols_all.total_bounds)],crs="EPSG:4326").explore()

## Extend grid

First we will load the existing grid:

In [18]:
existing_grid = utils.read_geojson_from_gcp("gs://ml4cc_data_lake/0_DEV/1_Staging/operational/grid_australia.geojson")
existing_grid_full_extent = gpd.GeoDataFrame(geometry=[box(*existing_grid.total_bounds)],crs="EPSG:4326")
existing_grid_full_extent.explore()

Secondly we create the new grid, we add an element to the grid if it's not in the previous grid (the intersection of the new grid element with the old grid is small).

In [7]:
step = 0.20, 0.20
size = 0.21

last_grid_name = max(existing_grid["name"])
grid_number = int(last_grid_name.replace("GRID",""))
grid_number+=1

bounds_inicial = (146.86329100021769, -37.50516044138397, 153.63872785102905, -23.244936010554465)

existing_grid_union = existing_grid.geometry.unary_union
pols_add = []
for x_left,y_bottom in itertools.product(np.arange(bounds_inicial[0]-10, bounds_inicial[2], step[0]),
                                         np.arange(bounds_inicial[1]-6.5,bounds_inicial[3]+13, step[1])):
    
    bounds_iter = (x_left, y_bottom, x_left+size, y_bottom+size)
    
    pol = box(*bounds_iter)
    
    intersection = pol.intersection(existing_grid_union)
    
    if (intersection.area/pol.area) >= .9:
        continue
    
    pols_add.append({"geometry": pol, "name":f"GRID{grid_number:05d}"})
    grid_number+=1
    

grid_add = gpd.GeoDataFrame(pols_add,crs="EPSG:4326")
# pols_all.to_file("/home/gonzalo/Downloads/grid_australia.geojson",driver="GeoJSON")  
# print(pols_all.shape)

m = gpd.GeoDataFrame(geometry=[box(*grid_add.total_bounds)],crs="EPSG:4326").explore()
existing_grid_full_extent.explore(m=m,color="red")

In [13]:
import pandas as pd
new_grid = pd.concat([existing_grid, grid_add], ignore_index=True)
assert new_grid.name.unique().shape[0] == new_grid.shape[0], "Grid names are not unique!!"
new_grid.shape

(14280, 2)

In [16]:
gpd.GeoDataFrame(geometry=[box(*new_grid.total_bounds)],crs="EPSG:4326").explore()

In [17]:
from ml4floods.data import utils
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/gonzalo/Downloads/ml4cc-general-access_request_pays.json"

utils.write_geojson_to_gcp("gs://ml4cc_data_lake/0_DEV/1_Staging/operational/grid_australia.geojson", new_grid)